# Assignment 1: Hidden Markov Model
### Author: Jacopo Raffi

In [35]:
import pandas as pd
import hmmlearn.hmm as hmm
import numpy as np

data = pd.read_csv("./data/AirQualityUCI.csv", sep=';')
data = data[['Date', 'Time', 'C6H6(GT)']] # for the assignment just need one sensor column

## Preprocessing data

In [ ]:
data = data.replace(',', '.', regex=True) # need to do this to convert numbers into float (values are written with ',' and not '.')
data['C6H6(GT)'] = data['C6H6(GT)'].astype(float)
data['C6H6(GT)'] = data['C6H6(GT)'].replace(-200, pd.NA) # so to avoid missing values when computing the averages

means = data.groupby('Time')['C6H6(GT)'].mean()
data['C6H6(GT)'] = data.apply(lambda row: means[row['Time']] if pd.isna(row['C6H6(GT)']) else row['C6H6(GT)'], axis=1)

seq = data['C6H6(GT)'].to_numpy()

## Gaussian HMM